In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import TextVectorization
import tqdm 
import os
import io

print(tf.__version__)



2.7.0


In [2]:
seed = 39
vocab_size = 100000
num_ns = 4
sequence_length = 25
embedding_dim = 300
batch_size = 1024
window_size = 2


In [3]:
# Generates skip-gram pairs with negative sampling for a list of sequences
# (int-encoded sentences) based on window size, number of negative samples
# and vocabulary size.
sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

def generate_training_data(sequence):
  # Elements of each training example are appended to these lists.
  targets, contexts, labels = [], [], []

  sequence = sequence.numpy()
 
  positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
          sequence,
          vocabulary_size=vocab_size,
          sampling_table=sampling_table,
          window_size=window_size,
          negative_samples=0)

  for target_word, context_word in positive_skip_grams:
    context_class = tf.expand_dims(
          tf.constant([context_word], dtype="int64"), 1)
    negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
          true_classes=context_class,
          num_true=1,
          num_sampled=num_ns,
          unique=True,
          range_max=vocab_size,
          seed=seed,
          name="negative_sampling")

      # Build context and label vectors (for one target word)
    negative_sampling_candidates = tf.expand_dims(
        negative_sampling_candidates, 1)

    context = tf.concat([context_class, negative_sampling_candidates], 0)
    label = tf.constant([1] + [0]*num_ns, dtype="int64")

      # Append each element from the training example to global lists.
    targets.append(target_word)
    contexts.append(context)
    labels.append(label)

  return targets, contexts, labels

In [4]:

conservative_ds = tf.data.TextLineDataset('conservative.txt')
liberal_ds = tf.data.TextLineDataset('liberal.txt')


text_ds = tf.data.Dataset.sample_from_datasets([conservative_ds, liberal_ds])

text_ds = text_ds.map(lambda x: tf.strings.split(x, maxsplit=1)[1]).batch(batch_size)


vectorize_layer = TextVectorization(
    max_tokens = vocab_size,
    output_mode = 'int', 
    output_sequence_length = sequence_length
)

vectorize_layer.adapt(text_ds)

inverse_vocab = vectorize_layer.get_vocabulary()
print(inverse_vocab[:60])





['', '[UNK]', 'the', 'trump', 'to', 'a', 'of', 'and', 'in', 'that', 'his', 'for', 'on', 'he', 'is', 'with', 'donald', 'said', 'as', 'president', 'has', 'was', 'at', 'by', 'it', 'who', 'from', 'be', 'not', 'but', 'an', 'have', 'about', 'would', 'had', 'will', 'campaign', 'this', 'are', 'i', 'full', 'or', 'him', 'text', 'new', 'after', 'they', 'if', 'one', 'more', 'when', 'house', 'republican', 'clinton', 'its', 'been', 'what', 'which', 'their', 'out']


In [5]:
text_vector_ds = text_ds.prefetch(tf.data.AUTOTUNE).map(vectorize_layer).unbatch()

In [21]:

def tf_generate_training_data(x):
    targets, contexts, labels = tf.py_function(generate_training_data, inp=[x], 
                                    Tout = [tf.int64, tf.int64, tf.int64])
    targets = tf.ensure_shape(targets, [targets.shape[0]])
    contexts = tf.ensure_shape(contexts, [targets.shape[0], num_ns+1, 1])
    labels = tf.ensure_shape(labels, [targets.shape[0], num_ns+1])

    return ((targets, contexts), labels)


skip_gram_ds = text_vector_ds.map(tf_generate_training_data, num_parallel_calls=tf.data.AUTOTUNE,       deterministic = False)
                                       


BATCH_SIZE = 1024
BUFFER_SIZE = 10000
AUTOTUNE = tf.data.AUTOTUNE


dataset = skip_gram_ds.unbatch()
dataset = dataset.cache().prefetch(buffer_size = AUTOTUNE)


dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print(dataset.element_spec)






((TensorSpec(shape=(1024,), dtype=tf.int64, name=None), TensorSpec(shape=(1024, 5, 1), dtype=tf.int64, name=None)), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None))


In [22]:
class Word2Vec(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()

        self.target_embedding = Embedding(vocab_size,
        embedding_dim,
        input_length=1,
        name="w2v_embedding"
        )

        self.context_embedding = Embedding(vocab_size,
        embedding_dim,
        input_length=num_ns+1
        )

    def call(self, pair):
         target, context = pair

         if len(target.shape) == 2:
            target = tf.squeeze(target, axis=1)

         word_emb = self.target_embedding(target)

         context_emb = self.context_embedding(context)

         context_emb = tf.squeeze(context_emb, axis = 2)

         dots = tf.einsum('be,bce->bc', word_emb, context_emb)

         return dots
        

In [23]:

word2Vec = Word2Vec(vocab_size, embedding_dim)
word2Vec.compile(optimizer = 'adam', 
                 loss = tf.keras.losses.CategoricalCrossentropy(from_logits = True),
                 metrics = ['accuracy'])

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="word2vec_logs")

word2Vec.fit(dataset, epochs=20, callbacks=[tensorboard_callback])

Epoch 1/20


InvalidArgumentError: 2 root error(s) found.
  (0) INVALID_ARGUMENT:  Shape of tensor EagerPyFunc:1 [0] is not compatible with expected shape [?,5,1].
	 [[{{node EnsureShape_1}}]]
	 [[IteratorGetNext]]
	 [[IteratorGetNext/_6]]
  (1) INVALID_ARGUMENT:  Shape of tensor EagerPyFunc:1 [0] is not compatible with expected shape [?,5,1].
	 [[{{node EnsureShape_1}}]]
	 [[IteratorGetNext]]
0 successful operations.
0 derived errors ignored. [Op:__inference_train_function_8752]

Function call stack:
train_function -> train_function


In [ ]:
weights = word2Vec.get_layer('w2v_embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

out_v = io.open('w2vVectors.tsv', 'w', encoding='utf-8')
out_m = io.open('w2Vmetadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [ ]:
print(len(word2Vec.get_layer('w2v_embedding').get_weights()[0]))

10000


In [ ]:
np.save('w2vVectors.npy', weights)